In [1]:
import os
import numpy as np
import pandas as pd


In [2]:
all_teams = pd.read_csv("mm_data\MDataFiles_Stage1\MTeams.csv")
reg_se_dt = pd.read_csv("mm_data\MDataFiles_Stage1\MRegularSeasonDetailedResults.csv")
tour_com = pd.read_csv("mm_data\MDataFiles_Stage1\MNCAATourneyCompactResults.csv")

In [3]:
def team_data_col(teamid, datas):
    #this function takes an integer input which stands for team id and exports the data information of the given team.  
    team_data = {'teamid' : teamid}
    team_winning_data = datas[datas.WTeamID == teamid]
    team_losing_data = datas[datas.WTeamID == teamid]
    team_data['win'] = team_winning_data.shape[0]
    team_data['los'] = team_winning_data.shape[0]
    team_data['played'] = team_data['los'] + team_data['win']
    n = team_data.get('played')
    team_data['score_avg'] = (team_losing_data.LScore.sum() + team_winning_data.WScore.sum())/n
    team_data['fgoal_avg'] = (team_winning_data.WFGM.sum() + team_losing_data.LFGM.sum())/n
    team_data['fgoal_at_avg'] = (team_winning_data.WFGA.sum() + team_losing_data.LFGA.sum())/n
    team_data['pointer_avg'] = (team_winning_data.WFGM3.sum() + team_losing_data.LFGM3.sum())/n
    team_data['pointer_at_avg'] = (team_winning_data.WFGA3.sum() + team_losing_data.LFGA3.sum())/n
    team_data['off_reb_avg'] = (team_winning_data.WOR.sum() + team_losing_data.LOR.sum())/n
    team_data['def_reb_avg'] = (team_winning_data.WDR.sum() + team_losing_data.LDR.sum())/n
    team_data['ast_avg'] = (team_winning_data.WAst.sum() + team_losing_data.LAst.sum())/n
    team_data['turn_avg'] = (team_winning_data.WTO.sum() + team_losing_data.LTO.sum())/n
    team_data['stl_avg'] = (team_winning_data.WStl.sum() + team_losing_data.LStl.sum())/n
    team_data['block_avg'] = (team_winning_data.WBlk.sum() + team_losing_data.LBlk.sum())/n
    team_data['foul_avg'] = (team_winning_data.WPF.sum() + team_losing_data.LPF.sum())/n
    return team_data

In [4]:
year_list0 = reg_se_dt.Season.unique()
year_list = []
for i in year_list0:
    year_list.append('year'+str(i))
data_by_year = {}
for i in range(len(year_list)):
    year_name = year_list0[i]
    reg_se_dt_yearly = reg_se_dt[reg_se_dt.Season == year_name]
    teams0 = np.concatenate([reg_se_dt_yearly.WTeamID.unique(),reg_se_dt_yearly.LTeamID.unique()])
    teams1 = np.unique(teams0)
    teams2 = teams1.tolist()
    all_teams_data = {}
    for j in teams2:
        all_teams_data[j] = team_data_col(j,reg_se_dt_yearly)
    data_by_year[year_name] = all_teams_data


C:\Users\aciha\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in longlong_scalars
  # Remove the CWD from sys.path while we load stuff.
C:\Users\aciha\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in longlong_scalars
  # This is added back by InteractiveShellApp.init_path()
C:\Users\aciha\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in longlong_scalars
  if sys.path[0] == '':
C:\Users\aciha\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in longlong_scalars
  del sys.path[0]
C:\Users\aciha\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in longlong_scalars
  
C:\Users\aciha\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in longlong_scalars
  from ipykernel import kernelapp as app
C:\Users\aciha\A

In [5]:
tour_com = tour_com.query('Season >= 2003 & Season != 2019')

In [6]:
games_df = pd.DataFrame(columns=['game_id', 'result', 'win_diff','loss_diff', 'played_diff', 'score_av_diff','fgoal_diff', 'fgoal_at_diff', 'pointer_diff','pointer_at_diff', 'off_reb_diff', 'def_reb_diff','ast_diff', 'turn_diff', 'stl_diff','block_diff', 'foul_diff'])
for i in range(len(tour_com)):
    year = tour_com.Season.iloc[i]
    win_id = tour_com.WTeamID.iloc[i]
    los_id = tour_com.LTeamID.iloc[i]
    win_data = data_by_year.get(year).get(win_id)
    loss_data = data_by_year.get(year).get(los_id)
    dice = np.round_(np.random.random(),decimals = 0)
    data_array = []
    if dice == 0:
        for key in win_data.keys():
            data_array.append(loss_data[key]-win_data[key])
        data_array[0] = 0    
    elif dice == 1:
        for key in win_data.keys():
            data_array.append(win_data[key]-loss_data[key])
        data_array[0] = 1
    data_array.insert(0,i)
    games_df.loc[len(games_df)] = data_array    

In [7]:
games_data = games_df.drop(['game_id','result'], axis = 1)
games_labels = games_df.result

In [8]:
from sklearn.linear_model import LogisticRegression
softmax_reg = LogisticRegression(multi_class = "multinomial", solver = "lbfgs", C = 1, max_iter = 1000)
softmax_reg.fit(games_data, games_labels)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [9]:
tour_com_test = pd.read_csv("mm_data\MDataFiles_Stage1\MNCAATourneyCompactResults.csv")
tour_com_test = tour_com_test.query('Season == 2019')

In [10]:
games_df_2019 = pd.DataFrame(columns=['game_id', 'result', 'win_diff','loss_diff', 'played_diff', 'score_av_diff','fgoal_diff', 'fgoal_at_diff', 'pointer_diff','pointer_at_diff', 'off_reb_diff', 'def_reb_diff','ast_diff', 'turn_diff', 'stl_diff','block_diff', 'foul_diff'])
for i in range(len(tour_com_test)):
    win_id = tour_com_test.WTeamID.iloc[i]
    los_id = tour_com_test.LTeamID.iloc[i]
    win_data = data_by_year.get(2019).get(win_id)
    loss_data = data_by_year.get(2019).get(los_id)
    data_array = []
    for key in win_data.keys():
        data_array.append(loss_data[key]-win_data[key])
    data_array[0] = 0    
    data_array.insert(0,i)
    games_df_2019.loc[len(games_df_2019)] = data_array    

In [11]:
games_df_2019_test = games_df_2019.drop(['game_id','result'], axis = 1)

In [12]:
sum(softmax_reg.predict(games_df_2019_test))/len(games_df_2019_test)

0.26865671641791045